In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BTC_ETH_LTC_1week_1m.csv')

In [3]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band


symbols = ['BTC', 'ETH', 'LTC']


nstd = 3

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)


df.dropna(inplace=True)

In [4]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = 1      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)
    
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        self.number_of_buys = len(self.buys)
        print('filled')

    def sell(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
        print('sold')



In [5]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD')
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD':  # Want to buy
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.buy(symbol, buy_price=df[f'{symbol}_Lower_Band'].iloc[i], time=df['OpenTime'].iloc[i])
                break

    if env.balance_unit != 'BUSD':  # Want to sell
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
            env.sell(sell_price=df[f'{symbol}_Upper_Band'].iloc[i], time=df['OpenTime'].iloc[i])

if env.balance_unit != 'BUSD':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
filled
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.0023668519761303576 BTC 1
0.002366851976